# MathDoku Solver

Similar to the puzzles published under the KenKen trademark.

## Testing the class

Have built a `MathDoku` class in [mathdoku](mathdoku.py). Testing with a backtracking solution here...


In [1]:
import mathdoku as md

In [2]:
grid_size = 4
cages = [(16, '*', 3), 
         (7, '+', 3), 
         (2, '-', 2),
         (12, '*', 3),
         (2, '/', 2),
         (2, '/', 2),
         (4, '=', 1)]
cage_map = [[0, 0, 1, 1],
              [2, 0, 1, 6],
              [2, 3, 4, 4],
              [3, 3, 5, 5]]

answer = [[2, 4, 1, 3],
          [1, 2, 3, 4],
          [3, 1, 4, 2],
          [4, 3, 2, 1]]


In [3]:
m = md.MathDoku(grid_size=grid_size)
m.init_puzzle(cages, cage_map)

In [4]:
for x in range(grid_size):
    for y in range(grid_size):
        m.set(x, y, answer[x][y])
print(m.as_grid())

2 4 1 3
1 2 3 4
3 1 4 2
4 3 2 1


In [5]:
def backtracker(puzzle, depth=0):
    if puzzle.num_empty_cells() == 0:
        return True
    
    x, y = puzzle.find_empty_cell()
    for val in puzzle.get_allowed_values(x, y):
        try:
            puzzle.set(x, y, val)
        except ValueError:
            continue

        if backtracker(puzzle, depth=depth+1):
            return True
        else:
            puzzle.clear(x, y)
    return False

In [6]:
m = md.MathDoku(grid_size=grid_size)
m.init_puzzle(cages, cage_map)

In [7]:
for i, cage in enumerate(m._cages):
    print(f"Cage {i}: {cage}; possibilities={m._cage_possibilities[i]}")

Cage 0: (16, <function product at 0x1124018c0>, 3); possibilities=[(1, 4, 4), (2, 2, 4)]
Cage 1: (7, <built-in function sum>, 3); possibilities=[(1, 2, 4), (1, 3, 3), (2, 2, 3)]
Cage 2: (2, <function subtract at 0x112420cb0>, 2); possibilities=[(3, 1), (4, 2)]
Cage 3: (12, <function product at 0x1124018c0>, 3); possibilities=[(1, 3, 4), (2, 2, 3)]
Cage 4: (2, <function divide at 0x112420560>, 2); possibilities=[(2, 1), (4, 2)]
Cage 5: (2, <function divide at 0x112420560>, 2); possibilities=[(2, 1), (4, 2)]
Cage 6: (4, <function equals at 0x112420710>, 1); possibilities=[(4,)]


In [38]:
backtracker(m)
print(m.as_grid())
m.is_solved()

2 4 1 3
1 2 3 4
3 1 4 2
4 3 2 1


True

## Exploratory Work

First question...given a target number, arithmetic operator, and range of digits, can we show all combinations that produce the right target?

Starting with addition and multiplication as they're associative and commutative.

In [9]:
max_digit = 4
all_digits = range(1, max_digit+1)

In [10]:
target = 7
operation = sum
num_digits = 3

In [11]:
# First, n choose k (e.g. 6 digits, choose 3)
import itertools
[x for x in itertools.combinations(all_digits, num_digits)]

[(1, 2, 3), (1, 2, 4), (1, 3, 4), (2, 3, 4)]

In [12]:
# Filter to just those that hit target
[x for x in itertools.combinations(all_digits, num_digits) if operation(x) == target]

[(1, 2, 4)]

In [13]:
import numpy
target = 24
operation = numpy.prod
[x for x in itertools.combinations(all_digits, num_digits) if operation(x)==target]

[(2, 3, 4)]

## First Complication

So far so good but there's a complication: if we use the rules from KenKen then digits can be repeated in a cage (as long as they're not also repeated in a row or column).


In [14]:
target = 16
[x for x in itertools.combinations(all_digits, num_digits) if operation(x)==target]

[]

In [15]:
2*2*4

16

It's OK though, because `itertools` has that covered in `combinations_with_replacement`.

In [16]:
[x for x in itertools.combinations_with_replacement(all_digits, num_digits) if operation(x)==target]

[(1, 4, 4), (2, 2, 4)]

## Second Complication

Next problem is that subtraction and division are not associative -- order changes the result. KenKen rules aren't particularly fussed about this, you decide the order in your head and that's fine. Generally the puzzles will only have 2 cells when the target must be produced with subtraction or division but apparently not all puzzles will do this.

In [17]:
def subtract(numbers):
    ret = numbers[0]
    for i in range(1, len(numbers)):
        ret -= numbers[i]
    return ret

def divide(numbers):
    ret = numbers[0]
    for i in range(1, len(numbers)):
        ret = ret // i
    return ret

So what we need to do here is, for each possible combination of digits, produce all the possible orderings of those digits. Because subtraction and division are not associative we'll need to use `product` instead of `combinations_with_replacement`. KenKen never uses negative targets in cages but some other puzzles do.

In [18]:
num_digits = 2
target = 2
operation = subtract

[x for x in itertools.product(all_digits, repeat=num_digits)]

[(1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4)]

In [19]:
[x for x in itertools.product(all_digits, repeat=num_digits) if operation(x) == target]

[(3, 1), (4, 2)]

# Representing MathDoku puzzles

These puzzles are a little more complicated to represent. We need:

* Standard N x N grid (latin squre)
* A list of "cages" -- target values and operations
* A mapping that tells us which cells are in which cages


In [20]:
import puzzlegrid as pg

grid = pg.ConstraintPuzzle(max_digit)

In [21]:
cages = [(16, numpy.product, 3), (7, sum, 3), (2, subtract, 2), (12, numpy.product, 3), (2, divide, 2), (2, divide, 2), (4, sum, 1)]
cage_cells = [[0, 0, 1, 1],
              [2, 0, 1, 6],
              [2, 3, 4, 4],
              [3, 3, 5, 5]]

In [22]:
grid.set(1, 3, 4)

In [23]:
print(grid)

.......4........


In [24]:
[x for x in range(max_digit, 1, -1)[0:2]]

[4, 3]

So let's get the possible values for each cage

In [25]:
cage_values = []
for i, c in enumerate(cages):
    target, operation, num = c
    combos = []
    if operation == sum or operation == numpy.product:
        combos = [x for x in itertools.combinations_with_replacement(all_digits, num)]
    else:
        combos = [x for x in itertools.product(all_digits, repeat=num)]
    cage_values.append([x for x in combos if operation(x) == target])
    print(i, cage_values[i])

0 [(1, 4, 4), (2, 2, 4)]
1 [(1, 2, 4), (1, 3, 3), (2, 2, 3)]
2 [(3, 1), (4, 2)]
3 [(1, 3, 4), (2, 2, 3)]
4 [(2, 1), (2, 2), (2, 3), (2, 4)]
5 [(2, 1), (2, 2), (2, 3), (2, 4)]
6 [(4,)]


We already have a value for cell 1, 3 (4). Let's see if that helps us with any of the other cells in that row or column.

In [26]:
for j in range(max_digit):
    print((1, j), grid.get_allowed_values(1, j), cage_values[cage_cells[1][j]])

(1, 0) {1, 2, 3} [(3, 1), (4, 2)]
(1, 1) {1, 2, 3} [(1, 4, 4), (2, 2, 4)]
(1, 2) {1, 2, 3} [(1, 2, 4), (1, 3, 3), (2, 2, 3)]
(1, 3) {4} [(4,)]


In [27]:
for i in range(max_digit):
    print((i, 3), grid.get_allowed_values(i, 3), cage_values[cage_cells[i][3]])

(0, 3) {1, 2, 3} [(1, 2, 4), (1, 3, 3), (2, 2, 3)]
(1, 3) {4} [(4,)]
(2, 3) {1, 2, 3} [(2, 1), (2, 2), (2, 3), (2, 4)]
(3, 3) {1, 2, 3} [(2, 1), (2, 2), (2, 3), (2, 4)]


Hmmm. So, basically, each cell can take any number from 1-4. 

This isn't going to work.

Better approach will be backtracking and not worrying about constraint propogation just yet.

One thought: take cell (0, 3). It has allowed values (1, 2, 3). Then the cage has some possible combinations, one of which includes "4" which we know is not allowed. So we should be able to reduce the list of allowed values?


In [28]:
row_and_col_allowed = {1, 2, 3}
cage_allowed = [(1, 2, 4), (1, 3, 3), (2, 2, 3)]

expect = [(1, 3, 3), (2, 2, 3)]

not_allowed = set(all_digits) - row_and_col_allowed

In [29]:
not_allowed

{4}

In [30]:
set(cage_allowed[0])

{1, 2, 4}

In [31]:
cage_allowed = [x for x in cage_allowed if not not_allowed & set(x)]
cage_allowed

[(1, 3, 3), (2, 2, 3)]

# Backtracking Solution

OK, might have enough now to attempt a backtracking solution...


In [32]:
def solve_backtracking(puzzle, depth=0):
    """Implements a simple "naive" back tracking solution"""
    if puzzle.num_empty_cells() == 0:
        return True

    x, y = puzzle.find_empty_cell()
    for val in puzzle.get_allowed_values(x, y):
        puzzle.set(x, y, val)
        if solve_backtracking(puzzle, depth=depth + 1):
            return True
        else:
            puzzle.clear(x, y)

    return False


In [33]:
solve_backtracking(grid)

True

In [34]:
print(grid)

1243213434124321


In [35]:
grid.is_puzzle_valid()

True

In [36]:
cell_map = [[] for x in range(len(cages))]
for i in range(max_digit):
    for j in range(max_digit):
        cell_map[cage_cells[i][j]].append((i, j))

cell_map

[[(0, 0), (0, 1), (1, 1)],
 [(0, 2), (0, 3), (1, 2)],
 [(1, 0), (2, 0)],
 [(2, 1), (3, 0), (3, 1)],
 [(2, 2), (2, 3)],
 [(3, 2), (3, 3)],
 [(1, 3)]]

In [37]:
for i, cells in enumerate(cell_map):
    target, operation, cell_count = cages[i]
    print(f"Checking cell {i}: Need a target {target} using {operation} from {cell_count} cells")
    cell_values = []
    for cell in cells:
        x, y = cell
        cell_values.append(grid.get(x, y))
    print(f"\tCell values in this cage are: {cell_values}")
    if operation(cell_values) == target:
        print(f"\tChecks out!")
    else:
        print(f"\tBOGUS cell values!")

Checking cell 0: Need a target 16 using <function product at 0x1133429e0> from 3 cells
	Cell values in this cage are: [1, 2, 1]
	BOGUS cell values!
Checking cell 1: Need a target 7 using <built-in function sum> from 3 cells
	Cell values in this cage are: [4, 3, 3]
	BOGUS cell values!
Checking cell 2: Need a target 2 using <function subtract at 0x11244c680> from 2 cells
	Cell values in this cage are: [2, 3]
	BOGUS cell values!
Checking cell 3: Need a target 12 using <function product at 0x1133429e0> from 3 cells
	Cell values in this cage are: [4, 4, 3]
	BOGUS cell values!
Checking cell 4: Need a target 2 using <function divide at 0x11244c7a0> from 2 cells
	Cell values in this cage are: [1, 2]
	BOGUS cell values!
Checking cell 5: Need a target 2 using <function divide at 0x11244c7a0> from 2 cells
	Cell values in this cage are: [2, 1]
	Checks out!
Checking cell 6: Need a target 4 using <built-in function sum> from 1 cells
	Cell values in this cage are: [4]
	Checks out!
